In [48]:
import torch 
import torchvision 
from torchvision import datasets
import progressbar
import os

In [49]:
path = "/Users/kibinda-moukenguemikhaelpresley/AMMI2021_Bootcamp_project/Zindi/Pytorch/"
os.chdir(path)

In [50]:
from dlc_practical_prologue import *

In [53]:
train_input, train_target, test_input, test_target = load_data(cifar = False, one_hot_labels=False, flatten=True, normalize = True)

* Using MNIST
Using downloaded and verified file: ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz
Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


HTTPError: HTTP Error 503: Service Unavailable

In [23]:
pwd

'/Users/kibinda-moukenguemikhaelpresley/AMMI2021_Bootcamp_project/Zindi/Pytorch'

## 1 - Activation Function

In [ ]:
class activ_funct():
    def __init__(self, x):
        self.x = x
        
    def sigma(self):
        x = torch.tensor(self.x)
        return (2 / (1 + np.exp(-2 * x))) - 1
    
    def dsigma(self):
        return 1 - self.sigma()**2

In [ ]:
# Example

sigma_dsigma = activ_funct([0,3, 5, 8, -3])

In [45]:
# 1 - Activation Function

def tan_h(x):
    return torch.tanh(x)

In [46]:
tan_h(torch.tensor(0))

tensor(0.)

In [47]:
def dtan_h(x):
    y = sigma(x)
    return 1 - torch.mul(y, y)

In [48]:
dtan_h(torch.tensor(0))

tensor(1.)

In [39]:
def tanh_(X):
    t = ((2 / (1 + np.exp(-2 * X))) - 1)
    return t

In [49]:
tanh_(torch.tensor([0, 2]))

tensor([0.0000, 0.9640], dtype=torch.float64)

In [41]:
def dtanh(X):
    dt = 1 - tanh_(X)**2
    return dt

In [50]:
dtanh(torch.tensor([0,2]))

tensor([1.0000, 0.0707], dtype=torch.float64)

In [9]:
# 2 - Loss Function

def loss(v, t):
    y = v-t
    return torch.sum(torch.mul(y, y))

In [10]:
def dloss(v, t):
    return 2 * (v-t)

In [11]:
# 3 - Forward and Backward Passes
def forward_pass(w1, b1, w2, b2, x):
    x0 = x
    s1 = torch.mm(w1, x0) + b1
    x1 = sigma(s1)
    s2 = torch.mm(w2, x1) + b2
    x2 = sigma(s2)
    return x0, s1, x1, s2, x2

In [12]:
def backward_pass(w1, b1, w2, b2, t, x, s1, x1, s2, x2, dl_dw1, dl_db1, dl_dw2, dl_db2):
    dl_dx2 = dloss(x2, t)
    dl_ds2 = torch.mul(dl_dx2, dsigma(s2))
    dl_dw2 += torch.mm(dl_ds2, torch.t(x1))
    dl_db2 += dl_ds2
    dl_dx1 = torch.mm(torch.t(w2), dl_ds2)
    dl_ds1 = torch.mul(dl_dx1, dsigma(s1))
    dl_dw1 += torch.mm(dl_ds1, torch.t(x))
    dl_db1 += dl_ds1

In [13]:
def compute_error(test_set, test_target_set, w1, b1, w2, b2):
    n = test_set.shape[0]
    num_errors = 0
    for i in range(n):
        x = test_set[i].resize(test_set[i].shape[0], 1)
        t = torch.argmax(test_target_set[i])
        _, _, _, _, predicted = forward_pass(w1, b1, w2, b2, x)
        predicted_class = torch.argmax(predicted, 0)
        if t != predicted_class:
            num_errors += 1
    return num_errors/n
